# Evaluate Models

## Imports

In [1]:
import os.path

import matplotlib.pyplot as plt

import numpy as np

import torch
from torch.utils.data import DataLoader

from torchvision import transforms

import yaml
import json

from model import RandomlyConnectedModel

from evaluation.hamlyn import evaluate_ssim
from evaluation.scared import evaluate_keyframes
from evaluation.utils import prepare_state_dict
from evaluation import sparsification as s
from evaluation import transforms as t

from loaders.hamlyn import HamlynDataset
from loaders.scared import SCAREDKeyframesLoader

## Configuration

In [11]:
model_name = 'uncertainty-l1.pt'
model_config = 'uncertainty-config.yml'

datasets_path = '../datasets/'
hamlyn_path = os.path.join(datasets_path, 'da-vinci')
scared_path = os.path.join(datasets_path, 'scared')

batch_size = 8
min_depth = 4.676 # mm
max_depth = 180.554 # mm

save_to = 'results'

In [9]:
device = torch.device('cuda') \
    if torch.cuda.is_available() \
    else torch.device('cpu')

print(f'Using CUDA? {"cuda" in repr(device)}')

with open(model_config) as f:
    config = yaml.load(f, Loader=yaml.Loader)

model = RandomlyConnectedModel(**config).to(device)

model_stem = os.path.splitext(model_name)[0]
model_save_to = os.path.join('results', model_stem)

hamlyn_transform = transforms.Compose([
    t.ResizeImage((256, 512)),
    t.ToTensor()
])

scared_transform = transforms.Compose([
    t.ResizeImage((1024, 1280)),
    t.ToTensor()
])

results_dict = {}

## Hamlyn SSIM/Sparsification

### Setup

In [13]:
hamlyn_model_path = os.path.join('models', 'hamlyn', model_name)
hamlyn_save_to = os.path.join(model_save_to, 'hamlyn')

state_dict = torch.load(hamlyn_model_path, map_location=device)
state_dict = prepare_state_dict(state_dict)

model.load_state_dict(state_dict)

hamlyn_dataset = HamlynDataset(hamlyn_path, 'test', hamlyn_transform)
hamlyn_dataloader = DataLoader(hamlyn_dataset, batch_size, drop_last=True)

### Run Hamlyn Evaluation

In [14]:
ssims, spars = evaluate_ssim(model, hamlyn_dataloader, hamlyn_save_to, device=device)

SSIM Evaluation:   0%|          | 0/899 [00:00<?, ?batch/s]/Users/louismanestar/Library/CloudStorage/OneDrive-ImperialCollegeLondon/MSc Artificial Intelligence/Master's Project/repos/model-evaluation/venv/lib/python3.8/site-packages/torch/nn/functional.py:4215: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
SSIM Evaluation:   1%|          | 7/899 [00:41<1:27:55,  5.91s/batch, left=0.622, right=0.604]


KeyboardInterrupt: 

#### SSIM Metric

In [ ]:
mean_ssim = np.mean(ssims)
std_ssim = np.std(ssims)

print('Hamlyn test set SSIM:'
      f'\n\t- Mean: {mean_ssim}'
      f'\n\t- Std. Deviation: {std_ssim}')

results_dict['ssim'] = {
    'mean': mean_ssim,
    'std': std_ssim
}

#### Sparsification Plot and Metrics

In [ ]:
pred_curves, oracle_curves, random_curves = zip(*spars)

pred_curve = np.array(pred_curves).mean(axis=0)
oracle_curve = np.array(oracle_curves).mean(axis=0)
random_curve = np.array(random_curves).mean(axis=0)

error_curve = s.sparsification_error(oracle_curve, pred_curve)

ause = s.ause(oracle_curve, pred_curve)
aurg = s.aurg(pred_curve, random_curve)

figure, (curve_axis, error_axis) = plt.subplots(2, 0)

x_axis = np.linspace(0, 1, num=len(oracle_curve))

curve_axis.plot(x_axis, oracle_curve, 'r-')
curve_axis.plot(x_axis, pred_curve)

error_axis.plot(x_axis, error_curve)

print('Hamlyn test set Sparsification:'
      f'\n\t- AUSE: {ause}'
      f'\n\t- AURG: {aurg}')

results_dict['sparsification'] = {
    'ause': ause,
    'aurg': aurg,
    'curves': {
        'oracle': oracle_curve,
        'predicted': pred_curve,
        'random': random_curve,
        'error': error_curve
    }
}

## SCARED MAE

### Setup

In [4]:
scared_model_path = os.path.join('models', 'hamlyn', model_name)
scared_save_to = os.path.join(model_save_to, 'scared')

state_dict = torch.load(scared_model_path, map_location=device)
state_dict = prepare_state_dict(state_dict)

model.load_state_dict(state_dict)

scared_dataset_8 = SCAREDKeyframesLoader(scared_path, 'test', 8, scared_transform)
scared_dataloader_8 = DataLoader(scared_dataset_8, 1)

print(f'Size of SCARED Dataset 8: {len(scared_dataset_8):,} images.')

scared_dataset_9 = SCAREDKeyframesLoader(scared_path, 'test', 9, scared_transform)
scared_dataloader_9 = DataLoader(scared_dataset_9, 1)

print(f'Size of SCARED Dataset 9: {len(scared_dataset_9):,} images.')

Size of SCARED Dataset 8: 5 images.
Size of SCARED Dataset 9: 5 images.


### Run SCARED Evaluation

In [5]:
maes_8 = evaluate_keyframes(model, scared_dataloader_8, min_depth, max_depth, device)
mean_mae_8 = sum(maes_8) / len(maes_8)
print(f'Dataset 8 Keyframe MAEs: {maes_8} mm')
print(f'Mean Absolute Depth on SCARED Dataset 8: {mean_mae_8} mm')

maes_9 = evaluate_keyframes(model, scared_dataloader_9, min_depth, max_depth, device)
mean_mae_9 = sum(maes_9) / len(maes_9)
print(f'Dataset 9 Keyframe MAEs: {maes_9} mm')
print(f'Mean Absolute Depth on SCARED Dataset 9: {mean_mae_9} mm')

results_dict['mae'] = {
    'dataset_8': mean_mae_8,
    'dataset_9': mean_mae_9
}

100%|██████████| 5/5 [00:23<00:00,  4.63s/batch, mae=tensor(50.6697, dtype=torch.float64)]


Dataset 8 Keyframe MAEs: [27.28447265338262, 25.89122560226983, 9.90304156224073, 65.7162050259811, 124.55368446595801] mm
Mean Absolute Depth on SCARED Dataset 8: 50.669725861966455 mm


100%|██████████| 5/5 [00:22<00:00,  4.41s/batch, mae=tensor(76.4943, dtype=torch.float64)] 

Dataset 9 Keyframe MAEs: [133.9774889871548, 21.60393080556474, 139.509304119105, 12.476071569642508, 74.90452305256555] mm
Mean Absolute Depth on SCARED Dataset 9: 76.4942637068065 mm


## Save Results

In [7]:
save_to_path = os.path.join(save_to, model_stem, 'evaluation.json')

with open(save_to_path, 'w+') as f:
    json.dump(results_dict, f, indent=4)